In [3]:
pip install pyspark


  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a9417ca6c89f7f9d5e9d01b977226fd1acd0fe49b38759f891837840c0870f96
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\51\c8\18\298a4ced8ebb3ab8a7d26a7198c0cc7035abb906bde94a4c4b
Successfully built pyspark


In [4]:
import findspark
findspark.init()
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark import SparkFiles


spark = SparkSession.builder \
       .master('local[*]') \
       .config("spark.driver.memory", "8g") \
       .config("spark.executor.memory", "2g") \
       .appName("nyc_taxi_parking_tickets") \
       .getOrCreate()

spark

In [6]:
df = spark.read.csv("Parking_Violations_Issued_-_Fiscal_Year_2017.csv",inferSchema=True, header= True)

In [7]:
# Get columns count
cols = len(df.columns)
print(f"DataFrame Columns count : {cols}")

# Get row count'
rows = df.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Columns count : 43
DataFrame Rows count : 10803028


In [8]:
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = true)
 |-- Vehicle Make: string (nullable = true)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = true)
 |-- Issuer Squad: string (nullable = true)
 |-- Violation Time: string (nullable = true)
 |-- Time First Observed: string (nullable = true)
 |-- Violation Coun

In [9]:
from pyspark.sql.functions import isnull, when, count, col
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).head()

Row(Summons Number=0, Plate ID=1, Registration State=0, Plate Type=0, Issue Date=0, Violation Code=0, Vehicle Body Type=42695, Vehicle Make=73047, Issuing Agency=0, Street Code1=0, Street Code2=0, Street Code3=0, Vehicle Expiration Date=0, Violation Location=2072400, Violation Precinct=0, Issuer Precinct=0, Issuer Code=0, Issuer Command=2062645, Issuer Squad=2063541, Violation Time=63, Time First Observed=9962281, Violation County=39547, Violation In Front Of Or Opposite=2161235, House Number=2288618, Street Name=4009, Intersecting Street=7435473, Date First Observed=0, Law Section=0, Sub Division=773, Violation Legal Code=8740321, Days Parking In Effect    =2712416, From Hours In Effect=5450946, To Hours In Effect=5450943, Vehicle Color=152332, Unregistered Vehicle?=9675432, Vehicle Year=0, Meter Number=9017555, Feet From Curb=0, Violation Post Code=3190187, Violation Description=1127470, No Standing or Stopping Violation=10803028, Hydrant Violation=10803028, Double Parking Violation=

In [10]:
dropping_columns = ['Time First Observed','Intersecting Street','Violation Legal Code','From Hours In Effect',
'To Hours In Effect','Unregistered Vehicle?','Meter Number','No Standing or Stopping Violation',
'Hydrant Violation','Double Parking Violation']
#dropping the columns
for i in dropping_columns:
  df = df.drop(i)

In [11]:
pip install -U scikit-learn

  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.1.3
    Uninstalling scikit-learn-1.1.3:
      Successfully uninstalled scikit-learn-1.1.3
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [13]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
from __future__ import print_function 
from pyspark.context import SparkContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.session import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,PCA
from pyspark.sql.functions import isnull, when, count, col,sum
#import packages to perform cross validation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import split,udf,col,regexp_replace,pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType,StringType,DoubleType
import pyspark.sql.functions as f
from collections import Counter
import pandas as pd
import numpy as np
import datetime 
import pickle
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [14]:
from pyspark.sql import functions as F

cols = ['Vehicle Body Type','Vehicle Make','Violation County','Violation In Front Of Or Opposite','Issuer Command','Issuer Squad',\
        'House Number','Street Name','Sub Division','Violation Description','Vehicle Color','Violation Post Code','Days Parking In Effect    ']
        
def calculate_mode(df, column_name):
    return df.groupBy(column_name).agg(F.count(column_name).alias("count"))\
        .sort(F.desc("count")).take(1)[0][0]

for column_name in cols:
    column_mode = calculate_mode(df, column_name)
    df = df.fillna(column_mode, subset=[column_name])

In [15]:
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).head()

Row(Summons Number=0, Plate ID=1, Registration State=0, Plate Type=0, Issue Date=0, Violation Code=0, Vehicle Body Type=0, Vehicle Make=0, Issuing Agency=0, Street Code1=0, Street Code2=0, Street Code3=0, Vehicle Expiration Date=0, Violation Location=2072400, Violation Precinct=0, Issuer Precinct=0, Issuer Code=0, Issuer Command=0, Issuer Squad=0, Violation Time=63, Violation County=0, Violation In Front Of Or Opposite=0, House Number=0, Street Name=0, Date First Observed=0, Law Section=0, Sub Division=0, Days Parking In Effect    =0, Vehicle Color=0, Vehicle Year=0, Feet From Curb=0, Violation Post Code=0, Violation Description=0)

In [16]:
df= df.na.drop()

In [17]:
df.printSchema()

root
 |-- Summons Number: long (nullable = true)
 |-- Plate ID: string (nullable = true)
 |-- Registration State: string (nullable = true)
 |-- Plate Type: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Violation Code: integer (nullable = true)
 |-- Vehicle Body Type: string (nullable = false)
 |-- Vehicle Make: string (nullable = false)
 |-- Issuing Agency: string (nullable = true)
 |-- Street Code1: integer (nullable = true)
 |-- Street Code2: integer (nullable = true)
 |-- Street Code3: integer (nullable = true)
 |-- Vehicle Expiration Date: integer (nullable = true)
 |-- Violation Location: integer (nullable = true)
 |-- Violation Precinct: integer (nullable = true)
 |-- Issuer Precinct: integer (nullable = true)
 |-- Issuer Code: integer (nullable = true)
 |-- Issuer Command: string (nullable = false)
 |-- Issuer Squad: string (nullable = false)
 |-- Violation Time: string (nullable = true)
 |-- Violation County: string (nullable = false)
 |-- Violation In

In [18]:
column_names = df.columns
print(column_names)

['Summons Number', 'Plate ID', 'Registration State', 'Plate Type', 'Issue Date', 'Violation Code', 'Vehicle Body Type', 'Vehicle Make', 'Issuing Agency', 'Street Code1', 'Street Code2', 'Street Code3', 'Vehicle Expiration Date', 'Violation Location', 'Violation Precinct', 'Issuer Precinct', 'Issuer Code', 'Issuer Command', 'Issuer Squad', 'Violation Time', 'Violation County', 'Violation In Front Of Or Opposite', 'House Number', 'Street Name', 'Date First Observed', 'Law Section', 'Sub Division', 'Days Parking In Effect    ', 'Vehicle Color', 'Vehicle Year', 'Feet From Curb', 'Violation Post Code', 'Violation Description']


In [19]:
new_columns = ['Summons Number','Feet From Curb',]
#dropping the columns
for j in new_columns:
  df= df.drop(j)

In [ ]:
from pyspark.ml.feature import StringIndexer
indexers = StringIndexer(inputCols= ['Plate ID','Registration State','Plate Type','Vehicle Body Type','Vehicle Make','Issuing Agency','Issuer Command',
                                     'Issuer Squad','Violation County','Violation In Front Of Or Opposite','House Number','Street Name','Sub Division','Days Parking In Effect    ',
                                     'Vehicle Color','Violation Time','Violation Post Code','Violation Description','Issue Date'],
                         outputCols=['PlateID_ind','RegistrationState_ind','PlateType_ind','VehicleBodyType_ind','VehicleMake_ind','IssuingAgency_ind','IssuerCommand_ind',
                                     'IssuerSquad_ind','ViolationCounty_ind','ViolationInFrontOfOrOpposite_ind','HouseNumber_ind','StreetName_ind','SubDivision_ind','DaysParkingInEffect_ind',
                                     'VehicleColor_ind','ViolationTime_ind','ViolationPostCode_ind','ViolationDescription_ind','IssueDate_ind'])
strindexedDF = indexers.fit(df).transform(df)

In [ ]:
for item in strindexedDF.head(1):
    print(item)
    print('\n')

In [19]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#creating vectors from features
#Apache MLlib takes input if vector form
assembler=VectorAssembler(inputCols=['PlateID_ind', 'RegistrationState_ind', 'PlateType_ind', 'IssueDate_ind', 
                                     'Violation Code', 'VehicleBodyType_ind', 'VehicleMake_ind', 'IssuingAgency_ind', 
                                     'Street Code1', 'Street Code2', 'Street Code3', 'Vehicle Expiration Date', 
                                     'Violation Precinct', 'Issuer Precinct', 'Issuer Code', 
                                     'IssuerCommand_ind', 'IssuerSquad_ind', 'ViolationTime_ind', 'ViolationCounty_ind', 
                                     'ViolationInFrontOfOrOpposite_ind', 'HouseNumber_ind', 'StreetName_ind', 'Date First Observed', 
                                     'Law Section', 'SubDivision_ind', 'DaysParkingInEffect_ind', 'VehicleColor_ind', 'Vehicle Year', 
                                     'ViolationPostCode_ind', 'ViolationDescription_ind'],outputCol='features')
output=assembler.transform(strindexedDF)
output.select('features','Violation Location').show(5)

+--------------------+------------------+
|            features|Violation Location|
+--------------------+------------------+
|[901.0,0.0,1.0,82...|                14|
|[2010.0,0.0,1.0,1...|                13|
|[12086.0,1.0,0.0,...|                71|
|[1291140.0,0.0,1....|               106|
|[117.0,0.0,1.0,41...|                18|
+--------------------+------------------+
only showing top 5 rows



In [20]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT

# Define a udf that converts sparse vector into dense vector
# You cannot create your own custom function and run that against the data directly. 
# In Spark, You have to register the function first using udf function
sparseToDense = F.udf(lambda v : Vectors.dense(v), VectorUDT())

# We then call the function here passing the column name on which the function has to be applied
densefeatureDF = output.withColumn('features_array', sparseToDense('features'))

densefeatureDF.select("features", "features_array").show(5, False)

Py4JJavaError: An error occurred while calling o925.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 60.0 failed 1 times, most recent failure: Lost task 0.0 in stage 60.0 (TID 350) (LAPTOP-1LK6U0FL executor driver): java.io.IOException: Cannot run program "python3": CreateProcess error=2, The system cannot find the file specified
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1128)
	at java.base/java.lang.ProcessBuilder.start(ProcessBuilder.java:1071)
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:167)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.ex

In [ ]:
# Import StandardScaler from pyspark.ml.feature package
from pyspark.ml.feature import StandardScaler

# Create the StandardScaler object. It only take feature column (dense vector)
stdscaler = StandardScaler(inputCol= "features", outputCol= "scaledfeatures")

# Fit the StandardScaler object on the output of the dense vector data and transform
stdscaledDF = stdscaler.fit(output).transform(output)
stdscaledDF.select("scaledfeatures" ).show(5, False)

In [ ]:
#final data consist of features and label which is crew.
final_data=stdscaledDF.select('features','Violation Location')
#splitting data into train and test
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

In [ ]:
test_data.describe().show()

In [ ]:
#import LinearRegression library
from pyspark.ml.regression import LinearRegression
#creating an object of class LinearRegression
#object takes features and label as input arguments
park_lr=LinearRegression(featuresCol='features',labelCol='Violation Location')
#pass train_data to train model
trained_parking_model=park_lr.fit(train_data)
#evaluating model trained for Rsquared error
park_results=trained_parking_model.evaluate(train_data)

In [ ]:
print('Rsquared Error :',park_results.r2)

In [ ]:
from pyspark.ml.feature import StringIndexer

# Apply StringIndexer to the label column
labelIndexer = StringIndexer(inputCol="Violation Location", outputCol="indexedLabel")
train_data = labelIndexer.fit(train_data).transform(train_data)


In [ ]:
from pyspark.ml.feature import VectorIndexer

indexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=32)
indexed_data = indexer.fit(train_data).transform(train_data)

In [ ]:
rf = RandomForestClassifier(numTrees=50, labelCol='indexedLabel', featuresCol="features")
rfmodel = rf.fit(train_data)
park_rf_results=rfmodel.evaluate(train_data)